# Segmenting and Clustering Neighborhoods in Toronto
----
## Question 1. Cleaning the Dataset

In [1]:
! pip install beautifulsoup4

In [2]:
import urllib.request #Importing a library to query webpage and return the html 

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'# url containg data of Toronto neighborhoods

In [4]:
html_doc=urllib.request.urlopen(url)# Querying the website to return html in html_doc

In [5]:
from bs4 import BeautifulSoup # Importing BeautifulSoup from bs4 to parse the html document html_doc

In [6]:
b_soup=BeautifulSoup(html_doc) # Parsing the html in html_doc variable

In [7]:
print(b_soup.prettify())# using prettify() function to look at the nested structure of the html document

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":900271985,"wgRevisionId":900271985,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

To find the table containing the neighborhood data of Toronto, we need to find the right table in the webpage.
We use the \< table \> tag to extract information within all table tags.

We find the table using class attribute.

In [8]:
data=b_soup.find('table',class_='wikitable sortable')

In [9]:
data #looking at the html structure of the table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

### Extracting information to a dataframe

#### Importing necessary libraries

In [10]:
import pandas as pd
import numpy as np

#### Finding the column names

In [11]:
# We observe that the name of the columns of the required dataframe is within <th> tag
columns=[]
for column in data.find_all('th'):
    columns.append(column.get_text())

In [12]:
columns

['Postcode', 'Borough', 'Neighbourhood\n']

#### Renaming columns

In [13]:
columns[0]='PostalCode'
columns[2]='Neighborhood'

In [14]:
columns

['PostalCode', 'Borough', 'Neighborhood']

Constructing the dataframe

In [15]:
df=pd.DataFrame(columns=columns)
df

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []

In [16]:
#We can access rows of the dataframe using the tr tag

### Extracting columns from the table in the lists A,B and C

In [17]:
A=[]
B=[]
C=[]
for row in data.find_all("tr"):
    elements=row.find_all('td')
    if len(elements)==3:
        A.append(elements[0].get_text().strip())
        B.append(elements[1].get_text().strip())
        C.append(elements[2].get_text().strip())
    
    

In [18]:
df['PostalCode']=A
df['Borough']=B
df['Neighborhood']=C

In [19]:
df.head()

PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront

#### Replacing 'Not assigned' in the dataframe with NaN object for easier analysis

In [20]:
df.replace('Not assigned',np.nan,inplace=True)# We replace Not assigned with in df

In [21]:
df.head()

PostalCode           Borough      Neighborhood
0        M1A               NaN               NaN
1        M2A               NaN               NaN
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront

#### We drop the rows with not assigned borough

In [22]:
df.dropna(subset=['Borough'],axis=0,inplace=True)

In [23]:
df.head()

PostalCode           Borough      Neighborhood
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M5A  Downtown Toronto       Regent Park
6        M6A        North York  Lawrence Heights

In [24]:
df.reset_index(drop=True,inplace=True)

In [25]:
df.head()

PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [26]:
df['Neighborhood']=df['Neighborhood'].replace(np.nan,df['Borough'])

In [27]:
df=df.astype(str)

In [28]:
df.shape

(211, 3)

#### Grouping neighborhoods with same Postal Code

In [29]:
df=df.groupby(df['PostalCode'],sort=False).agg(lambda x: ', '.join(set(x))).reset_index()

In [30]:
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A      Queen's Park                      Queen's Park

#### Shape of the modified dataframe

In [31]:
df.shape

(103, 3)

In [32]:
df.to_csv('TorontoDataset.csv')